## Function that reads all CSVs and concatenates them
This file is designed to process historical Forex data stored in CSV files. The code searches for all CSV files in a specified directory, reads each file into a GPU-accelerated DataFrame using cudf, converts the 'date' column into a datetime format, and sets it as the index. Finally, it concatenates all individual DataFrames into a single DataFrame and saves the results as a parquet file for efficient storage and further analysis.

In [1]:
import cudf as cf
import os

def load_and_concat_csvs(folder_path):
    csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    dfs = []
    for file in csv_files:
        df_temp = cf.read_csv(file, sep=';', names=['date', 'open', 'high', 'low', 'close', 'volume'])
        df_temp['date'] = cf.to_datetime(df_temp['date'], format='%Y%m%d %H%M%S%f')
        df_temp.set_index('date', inplace=True)
        dfs.append(df_temp)
    return cf.concat(dfs)

# Example usage: concatenate all CSV files in the folder
eurgbp_merged = load_and_concat_csvs('/home/edoardocame/Desktop/python_dir/data/forexhistdata')

eurgbp_merged.to_parquet('/home/edoardocame/Desktop/python_dir/data/forexhistdata.parquet')

The `dfs` list is used to store all the DataFrames created from reading each CSV file. Here's a step-by-step explanation of the code:

1. **List CSV Files**: 
   ```python
   csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
   ```
   This line creates a list of file paths for all CSV files in the specified `folder_path`.

2. **Initialize List**:
   ```python
   dfs = []
   ```
   An empty list `dfs` is initialized to store the DataFrames.

3. **Read and Process Each CSV**:
   ```python
   for file in csv_files:
       df_temp = cf.read_csv(file, sep=';', names=['date', 'open', 'high', 'low', 'close', 'volume'])
       df_temp['date'] = cf.to_datetime(df_temp['date'], format='%Y%m%d %H%M%S')
       df_temp.set_index('date', inplace=True)
       dfs.append(df_temp)
   ```
   - For each CSV file, it reads the file into a DataFrame `df_temp`.
   - Converts the 'date' column to datetime format.
   - Sets the 'date' column as the index.
   - Appends the DataFrame `df_temp` to the `dfs` list.

4. **Concatenate DataFrames**:
   ```python
   return cf.concat(dfs)
   ```
   Finally, it concatenates all the DataFrames in the `dfs` list into a single DataFrame and returns it.

So, `dfs` is indeed a list containing all the DataFrames created from each CSV file.